In [1]:
import logging
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yaml
import openpyxl
from openpyxl import load_workbook
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter
import argparse
import sys
from typing import List, Dict

def main(config_path: str):
    """
    Main function to load configuration, preprocess data, create charts, and save Excel file.
    """
    config = load_config(config_path)
    validate_config(config)
    data = load_data(config['file_path'])
    data = preprocess_data(data)
    output_folder = os.path.join(config['output_folder'], f"{config['unidade']}_{config['periodo']}")
    os.makedirs(output_folder, exist_ok=True)
    create_charts(data, output_folder, config['unidade'], config['periodo'], config)
    save_copy_of_source_file_as_excel(data, output_folder, config['unidade'], config['periodo'], config)

# Configurar o logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_config() -> Dict:
    """
    Loads configuration from a YAML file located at a specific path.

    Returns:
        dict: Configuration data.
    """
    config_path = "/home/roni-chittoni/ProjectsHub/GAPD/core/laboratorio/config/yaml_files/monitor_colheita.yaml"
    
    # Verifica se o arquivo existe antes de tentar carregar
    if not os.path.exists(config_path):
        logger.error(f"Configuration file not found at {config_path}")
        raise FileNotFoundError(f"Configuration file not found at {config_path}")

    try:
        with open(config_path, 'r') as file:
            config = yaml.safe_load(file)
        logger.info(f"Configuration loaded successfully from {config_path}")
        return config
    except yaml.YAMLError as e:
        logger.error(f"Error parsing YAML file: {e}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error loading configuration: {e}")
        raise

def validate_config(config: Dict):
    """
    Validates the configuration to ensure all required fields are present.

    Args:
        config (dict): Configuration data.

    Raises:
        ValueError: If any required field is missing.
    """
    required_fields = ['file_path', 'stacked_chart_variables', 'violin_plot_columns', 'output_folder',
                       'unidade', 'periodo', 'stacked_chart_titles', 'violin_plot_title', 'violin_plot_x_labels',
                       'violin_plot_x_axis_label', 'violin_plot_y_axis_label', 'excel_columns_to_drop', 
                       'excel_conditional_formatting_columns']
    missing_fields = [field for field in required_fields if field not in config]
    if missing_fields:
        raise ValueError(f"Missing required configuration fields: {missing_fields}")
    logger.info("Configuration validated successfully")

def load_data(file_path: str) -> pd.DataFrame:
    """
    Loads data from a CSV file.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        pd.DataFrame: Loaded data.
    """
    try:
        data = pd.read_csv(file_path)
        logger.info(f"Data loaded successfully from {file_path}")
        return data
    except FileNotFoundError:
        logger.error(f"File not found: {file_path}")
        raise
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        raise

def preprocess_data(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesses data by converting dates and reordering columns.

    Args:
        data (pd.DataFrame): Data to preprocess.

    Returns:
        pd.DataFrame: Preprocessed data.
    """
    data = convert_dates(data)
    data = reorder_columns(data)
    logger.info("Data preprocessed successfully")
    return data

def convert_dates(data: pd.DataFrame) -> pd.DataFrame:
    """
    Converts UTC dates to BRT.

    Args:
        data (pd.DataFrame): Data containing the 'Submit Date (UTC)' column.

    Returns:
        pd.DataFrame: Data with converted dates.
    """
    data['Submit Date (UTC)'] = pd.to_datetime(data['Submit Date (UTC)']).dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    data['Submit Date (UTC)'] = data['Submit Date (UTC)'].dt.tz_localize(None)
    data.rename(columns={'Submit Date (UTC)': 'Submit Date (BRT)'}, inplace=True)
    return data

def reorder_columns(data: pd.DataFrame) -> pd.DataFrame:
    """
    Reorders columns to place 'Submit Date (BRT)' first.

    Args:
        data (pd.DataFrame): Data to reorder columns.

    Returns:
        pd.DataFrame: Data with reordered columns.
    """
    cols = list(data.columns)
    cols.insert(0, cols.pop(cols.index('Submit Date (BRT)')))
    data = data[cols]
    return data

def create_stacked_count_chart(data: pd.DataFrame, output_folder: str, unidade: str, periodo: str, column: str, title: str):
    """
    Creates and saves a stacked bar chart.

    Args:
        data (pd.DataFrame): Data for the chart.
        output_folder (str): Folder to save the chart.
        unidade (str): Unit name.
        periodo (str): Period.
        column (str): Column for the stacked chart.
        title (str): Title for the chart.
    """
    plt.figure(figsize=(10, 6))
    counts = data.groupby(['Seu nome', column]).size().unstack(fill_value=0)

    # Define the specific colors for notes 1 to 5
    color_mapping = {
        1: '#ff9999',  # Red
        2: '#ffcc99',  # Orange
        3: '#ffff99',  # Yellow
        4: '#99ff99',  # Green
        5: '#99ccff'   # Blue
    }
    
    # Get the unique column values (notes) and map them to colors
    unique_columns = counts.columns
    colors = [color_mapping[note] for note in unique_columns if note in color_mapping]

    # Check if the number of colors matches the number of columns
    if len(colors) != len(unique_columns):
        raise ValueError("Number of defined colors does not match the number of unique categories.")

    ax = counts.plot(kind='bar', stacked=True, color=colors)

    for container in ax.containers:
        labels = [f'{int(v)}' if v > 0 else '' for v in container.datavalues]
        ax.bar_label(container, labels=labels, label_type='center', fontsize=8)

    plt.title(f'{title} - {periodo}')
    plt.xlabel('Seu nome')
    plt.ylabel('Counts')
    plt.tight_layout()
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='lightgrey')
    ax.set_axisbelow(True)
    output_path = os.path.join(output_folder, f'stacked_bar_{column}.png')
    plt.savefig(output_path)
    plt.close()
    logger.info(f"Stacked bar chart for {column} saved at {output_path}")

def filter_numeric_columns(data: pd.DataFrame, numeric_columns: List[str]) -> pd.DataFrame:
    """
    Filters numeric columns from the data.

    Args:
        data (pd.DataFrame): Data to filter.
        numeric_columns (list): List of numeric column names.

    Returns:
        pd.DataFrame: Data with only numeric columns.
    """
    return data[numeric_columns]

def create_violin_plot(data: pd.DataFrame, unidade: str, periodo: str, config: Dict) -> str:
    """
    Creates a violin plot.

    Args:
        data (pd.DataFrame): Data for the plot.
        unidade (str): Unit name.
        periodo (str): Period.
        config (dict): Configuration data.

    Returns:
        str: Filename of the saved plot.
    """
    plt.figure(figsize=(10, 6))
    ax = sns.violinplot(data=data, orient='h')
    ax.set_xticks([1, 2, 3, 4, 5])
    ax.set_xticklabels(config['violin_plot_x_labels'])
    plt.title(f"{config['violin_plot_title']} - {unidade} - {periodo}")
    plt.xlabel(config['violin_plot_x_axis_label'])
    plt.ylabel(config['violin_plot_y_axis_label'])
    plt.tight_layout()
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='lightgrey')
    ax.set_axisbelow(True)
    filename = f'violin_plot_{unidade}_{periodo}.png'
    return filename

def save_plot(filename: str, output_folder: str):
    """
    Saves the current plot to a file.

    Args:
        filename (str): Filename to save the plot.
        output_folder (str): Folder to save the plot.
    """
    output_path = os.path.join(output_folder, filename)
    plt.savefig(output_path)
    plt.close()
    logger.info(f"Violin plot saved at {output_path}")

def apply_conditional_formatting(ws, columns: List[str]):
    """
    Applies conditional formatting to specified columns in an Excel worksheet.

    Args:
        ws (openpyxl.worksheet.worksheet.Worksheet): Worksheet to format.
        columns (list): List of column letters to apply formatting.
    """
    for col in columns:
        c_range = f"{col}2:{col}{ws.max_row}"
        rule = ColorScaleRule(start_type='num', start_value=1, start_color='FFB3BA',
                              mid_type='num', mid_value=3, mid_color='FFDFBA',
                              end_type='num', end_value=5, end_color='BAE1FF')
        ws.conditional_formatting.add(c_range, rule)
        for row in ws[c_range]:
            for cell in row:
                cell.alignment = Alignment(horizontal='center')

def save_copy_of_source_file_as_excel(data: pd.DataFrame, output_folder: str, unidade: str, periodo: str, config: Dict):
    """
    Saves a copy of the source data as an Excel file with conditional formatting.

    Args:
        data (pd.DataFrame): Data to save.
        output_folder (str): Folder to save the file.
        unidade (str): Unit name.
        periodo (str): Period.
        config (dict): Configuration data.
    """
    columns_to_drop = config['excel_columns_to_drop']
    data = data.drop(columns=columns_to_drop, errors='ignore')
    base_name = f'{unidade}_{periodo}'
    new_file_path = os.path.join(output_folder, f'{base_name}.xlsx')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    with pd.ExcelWriter(new_file_path, engine='openpyxl') as writer:
        data.to_excel(writer, index=False, sheet_name='Sheet1')
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']
        worksheet.auto_filter.ref = worksheet.dimensions
    wb = load_workbook(new_file_path)
    ws = wb.active
    columns = config['excel_conditional_formatting_columns']
    column_letters = [get_column_letter(data.columns.get_loc(col) + 1) for col in columns]
    apply_conditional_formatting(ws, column_letters)
    wb.save(new_file_path)
    print(f'Cópia do arquivo de origem salva em: {new_file_path}')

def create_charts(data: pd.DataFrame, output_folder: str, unidade: str, periodo: str, config: Dict):
    """
    Creates and saves charts based on the configuration.

    Args:
        data (pd.DataFrame): Data for the charts.
        output_folder (str): Folder to save the charts.
        unidade (str): Unit name.
        periodo (str): Period.
        config (dict): Configuration dictionary.
    """
    stacked_chart_columns = config['stacked_chart_variables']
    for column in stacked_chart_columns:
        title = config['stacked_chart_titles'][column]
        create_stacked_count_chart(data, output_folder, unidade, periodo, column, title)
    numeric_columns = config['violin_plot_columns']
    data_numeric = filter_numeric_columns(data, numeric_columns)
    filename = create_violin_plot(data_numeric, unidade, periodo, config)
    save_plot(filename, output_folder)

if __name__ == "__main__":
    if 'ipykernel' in sys.argv[0]:
        # Executado no Jupyter Notebook
        config_path = 'monitor_colheita.yaml'
    else:
        # Executado no terminal
        parser = argparse.ArgumentParser(description='Survey Response Analysis')
        parser.add_argument('--config', type=str, default='monitor_colheita.yaml', help='Path to the configuration file')
        args = parser.parse_args()
        config_path = args.config
    
    main(config_path)

TypeError: load_config() takes 0 positional arguments but 1 was given